In [72]:
from jaqs.data import RemoteDataService, DataView

data_config = {
    "remote.data.address": "tcp://data.tushare.org:8910",
    "remote.data.username": "phone",
    "remote.data.password": "token"
}

UNIVERSE = '000300.SH'

start_date = 20171201
curr_date = 20171212

dataview_props = {# Start and end date of back-test
                  'start_date': start_date, 
                  'end_date': curr_date,
                  # Investment universe and performance benchmark
                  'universe': UNIVERSE, 
                  'benchmark': '000300.SH',
                  # Data fields that we need
                  'fields': 'close,open,high,low',
                  # freq = 1 means we use daily data. Please do not change this.
                  'freq': 1}

# RemoteDataService communicates with a remote server to fetch data
ds = RemoteDataService()
# Use username and password in data_config to login
ds.init_from_config(data_config)

# DataView utilizes RemoteDataService to get various data and store them
dv = DataView()
dv.init_from_config(dataview_props, ds)
dv.prepare_data()

# add formula to calculate three yang k line
dv.add_formula('is_yang', '(close > open) && ( (close - open) / (high - low) >= 0.45)', is_quarterly=False)
dv.add_formula('three_yang', 'is_yang && Delay(is_yang, 1) && Delay(is_yang, 2)', is_quarterly=False)

# get the result
df = dv.get_snapshot(curr_date)
df[df['three_yang'] == 1.0]


Begin: DataApi login 18612562791@tcp://data.tushare.org:8910
    Already login as 18612562791, skip init_from_config
Initialize config success.
Query data...
Query data - query...
NOTE: price adjust method is [post adjust]
Query data - daily fields prepared.
Query instrument info...
Query adj_factor...
Query benchmark...
Query benchmar member info...
Data has been successfully prepared.


field,adjust_factor,close,close_adj,high,high_adj,index_member,index_weight,is_yang,low,low_adj,open,open_adj,three_yang,trade_status,vwap,vwap_adj
symbol,,,,,,,,,,,,,,,,
002304.SZ,3.925653,112.48,112.48,113.57,113.57,1,0.005577,1.0,110.56,110.56,111.02,111.02,1.0,交易,112.50,112.50
600606.SH,119.400316,7.31,7.31,7.34,7.34,1,0.002334,1.0,7.22,7.22,7.25,7.25,1.0,交易,7.28,7.28
603799.SH,1.006406,90.81,90.81,92.50,92.50,1,NaN,1.0,85.80,85.80,86.00,86.00,1.0,交易,89.92,89.92
